In [1]:
import pandas as pd 
import altair as alt 

import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerTuple
from collections import defaultdict

import plotly.graph_objects as go
import plotly.io as pio

In [2]:
def draw_line(df, experiment_type, pvalue_threshold = 0.05):
    df = df.copy()
    df[f"pvalue lower than {pvalue_threshold}"] = df["pvalue"] <= pvalue_threshold
    sign_df = df.groupby(["M","sample_size"])[f"pvalue lower than {pvalue_threshold}"].aggregate(["sum", "count"]).reset_index()
    sign_df["Percentage"] = sign_df["sum"] / sign_df["count"]
    sign_df["Experiment type"] = experiment_type

    base = alt.Chart(sign_df)
    line = base.mark_line() + base.mark_circle()
    # "#66C2A5" "#FC8D62" "#8DA0CB" "#E78AC3" "#A6D854" "#FFD92F" "#E5C494" "#B3B3B3"
    line = line.encode(
        alt.X("M:O", axis=alt.Axis(labelAngle=0)),
        alt.Y("Percentage", scale=alt.Scale(domain=[0, 1.0]), title="Percentage"),
        alt.Color("sample_size:O", 
                  title="SEED design Sample size",
                  scale=alt.Scale(scheme="set2", 
                                  domain=[18,24,32,48],   
                                  range=["#66C2A5", "#FC8D62", "#8DA0CB", "#E78AC3"])
        ),
        # strokeDash="Experiment type:O"
    )
    rule = base.mark_rule(color="red").encode(
        # alt.Y("sum(pct):Q", scale=alt.Scale(domain=[0, 1.0])).title("Percentage"),
        alt.Y("threshold:Q", title="Percentage"),
    )
    chart = alt.layer(line, rule).transform_calculate(
        threshold="0.8"
    ).properties(
        width=500,
        height=250,
    )
    return chart


def draw_figure(folder_name, biomarker, ratio=100):
    df = pd.read_csv(f"./{folder_name}/statistic_estimation_crossover_{biomarker}_{ratio}.csv")
    chart = draw_line(df, experiment_type="Crossover design", pvalue_threshold=0.05)
    
    df = pd.read_csv(f"./{folder_name}/statistic_estimation_parallel_{biomarker}_{ratio}.csv")
    df = df[df["sample_size"].isin([36,48,64,96])]
    pvalue_threshold = 0.05
    df[f"pvalue lower than {pvalue_threshold}"] = df["pvalue"] <= pvalue_threshold
    sign_df = df.groupby(["M","sample_size", "sample_size_pergroup"])[f"pvalue lower than {pvalue_threshold}"].aggregate(["sum", "count"]).reset_index()
    sign_df["Percentage"] = sign_df["sum"] / sign_df["count"]
    sign_df["Experiment type"] = "Parallel design"
    rule = alt.Chart(sign_df).mark_rule(strokeDash=(10,10), strokeWidth=2, color="#8c564b").encode(
        alt.Y("Percentage:Q"),
        alt.StrokeDash("sample_size:O", title="Parallel design Sample size", )
    )
    chart = (chart + rule).properties(
        title=f"{biomarker}: within-subject variance = {ratio}% * between-subject variance"
    )
    return chart

In [3]:
biomarker = "pTau217"
# folder_name = "outputs_e1.0_m55"
# folder_name = "outputs_e1.0_m33"
folder_name = "./archived_simulations/outputs_e1.0_m44"
# folder_name = "outputs_e0.25_m55"

In [4]:
# chart = draw_figure(biomarker=biomarker, folder_name=folder_name, ratio=100)
# chart.save("figs/100percent_ratio.png", ppi=200) # scale_factor=4) 
# 
# chart = draw_figure(biomarker=biomarker, folder_name=folder_name, ratio=60)
# chart.save("figs/60percent_ratio.png", ppi=200) # scale_factor=4)
# 
# chart = draw_figure(biomarker=biomarker, folder_name=folder_name, ratio=30)
# chart.save("figs/30percent_ratio.png", ppi=200) # scale_factor=4)

In [5]:
# def concat_data_lmer(folder_name, biomarker, ratio=30, pvalue_threshold=0.05):
#     df_cross = pd.read_csv(f"./{folder_name}/statistic_estimation_crossover_{biomarker}_{ratio}.csv")
#     df_cross = df_cross.drop("M", axis=1)
#     df_cross[f"pvalue lower than {pvalue_threshold}"] = df_cross["pvalue"] <= pvalue_threshold
#     df_cross = df_cross.groupby(["sample_size"])[f"pvalue lower than {pvalue_threshold}"].aggregate(["sum", "count"]).reset_index()
#     df_cross["Percentage"] = df_cross["sum"] / df_cross["count"]
#     df_cross["Experiment type"] = "SEED design"
#     
#     df_paral = pd.read_csv(f"./{folder_name}/statistic_estimation_parallel_{biomarker}_{ratio}.csv")
#     df_paral[f"pvalue lower than {pvalue_threshold}"] = df_paral["pvalue"] <= pvalue_threshold
#     df_paral = df_paral.groupby(["sample_size"])[f"pvalue lower than {pvalue_threshold}"].aggregate(["sum", "count"]).reset_index()
#     df_paral["Percentage"] = df_paral["sum"] / df_paral["count"]
#     df_paral["Experiment type"] = "Parallel design"
#     df = pd.concat([df_cross, df_paral])
#     df["Ratio"] = ratio / 100
#     return df


def concat_data_lmer(folder_name, biomarker, ratio=30, pvalue_threshold=0.05):
    df_cross = pd.read_csv(f"./{folder_name}/statistic_estimation_crossover_{biomarker}_{ratio}.csv")
    df_cross = df_cross.drop("M", axis=1)
    df_cross[f"pvalue lower than {pvalue_threshold}"] = (df_cross["pvalue"] <= pvalue_threshold) & (df_cross["param"] < 0)
    df_cross = df_cross.groupby(["sample_size"])[f"pvalue lower than {pvalue_threshold}"].aggregate(["sum", "count"]).reset_index()
    df_cross["Percentage"] = df_cross["sum"] / df_cross["count"]
    df_cross["Experiment type"] = "SEED design"
    
    df_paral = pd.read_csv(f"./{folder_name}/statistic_estimation_parallel_{biomarker}_{ratio}.csv")
    df_paral[f"pvalue lower than {pvalue_threshold}"] = (df_paral["pvalue"] <= pvalue_threshold) & (df_paral["param"] < 0)
    df_paral = df_paral.groupby(["sample_size"])[f"pvalue lower than {pvalue_threshold}"].aggregate(["sum", "count"]).reset_index()
    df_paral["Percentage"] = df_paral["sum"] / df_paral["count"]
    df_paral["Experiment type"] = "Parallel design"
    df = pd.concat([df_cross, df_paral])
    df["Ratio"] = ratio / 100
    return df

dfs = [concat_data_lmer(biomarker=biomarker, folder_name=folder_name, ratio=i*10) for i in range(1,11)]
df = pd.concat(dfs)
df["Experiment type"].unique()

array(['SEED design', 'Parallel design'], dtype=object)

In [6]:
fig = go.Figure()

colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#17becf"]
sample_sizes = [18, 24, 32, 36, 48, 64, 96, 112, 136]
exp_types = ["Crossover design", "Parallel design"]

config_pairs = [
    ("SEED design",      18, "#1f77b4"),
    ("Parallel design",  36, "#1f77b4"),
    ("SEED design",      24, "#ff7f0e"),
    ("Parallel design",  48, "#ff7f0e"),
    ("SEED design",      32, "#2ca02c"),
    ("Parallel design",  64, "#2ca02c"),
    ("SEED design",      48, "#7f7f7f"),
    ("Parallel design",  96, "#7f7f7f"),
    ("Parallel design", 112, "#8c564b"),
    ("Parallel design", 130, "#e377c2"),
    ("Parallel design", 136, "#17becf"),
]


for exp_type, sample_size, color in config_pairs:
    data = df[(df["sample_size"] == sample_size) & (df["Experiment type"] == exp_type)]
    if data.empty:
        continue

    linestyle = "dash" if exp_type == "Parallel design" else None
    label = f"{exp_type} ({sample_size})"  # Combine design and sample size
    if linestyle:
        fig.add_trace(go.Scatter(x=data["Ratio"], y=data["Percentage"], name=label,
                                 line=dict(color=color, width=2,
                                      dash='dash') # dash options include 'dash', 'dot', and 'dashdot'
        ))
    else:
        fig.add_trace(go.Scatter(x=data["Ratio"], y=data["Percentage"], name=label,
                                 line=dict(color=color, width=2) 
        ))
fig.add_hline(y=0.8, line=dict(color="red", width=1))
# Edit the layout
fig.update_layout(
        title=dict(
            text='pTau217 Power Estimation'
        ),
        xaxis=dict(
            title='Within-subject and Between-subject Variance Ratio',
            tickmode = 'linear',
            tick0 = 0.0,
            dtick = 0.1,
        ),
        yaxis=dict(
            title='Power',
            tickformat='.0%',
            tick0 = 0.0,
            dtick = 0.1,
            range = [0,1.05]
        ),
        width=1000,
        height=500,
)
pio.write_image(fig, './figs/multiratio_simulation.png',scale=4, width=1000, height=500)
fig.show()